In [2]:
import pandas as pd
import numpy as np
import os


def P(df, col):
    """
    Calculate the probability of each unique value in a column of a DataFrame and return a mapping the value to its probability.
    """
    # Calculate the probability of each unique value in the column
    prob = df[col].value_counts(normalize=True).to_dict()
    # Create a mapping of the value to its probability
    mapping = {k: v for k, v in prob.items()}
    return mapping

df = pd.read_csv(f'{os.getcwd()}/cleaned_data.csv')
df

C:\Users\franj\AppData\Local\Temp\ipykernel_31148\539358981.py:16: DtypeWarning: Columns (1,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{os.getcwd()}/cleaned_data.csv')


,ship_company,train_operator,container_type,dangerous_goods,custom,waste,full_empty,content,weight,train_id,...,month_of_arrival,container_owner_prefix,equipment_identifier,weather_today,weather_prev_day,weather_next_day_1,weather_next_day_2,weather_next_day_3,weather_next_day_4,weather_next_day_5
0,MAE-HAM,BOX,20G,0,0,0,1,NaN,2826.0,BRV13837Z,...,January,HAS,U,Regen,Niesel,Unwetter,Niesel,Schnee,Schnee,Schnee
1,MAE-HAM,BOX,40GH,0,0,0,1,NaN,7331.0,BRV13837Z,...,January,MSK,U,Regen,Niesel,Unwetter,Niesel,Schnee,Schnee,Schnee
2,MAE-HAM,BOX,40GH,0,0,0,1,NaN,8815.0,BRV13837Z,...,January,GES,U,Regen,Niesel,Unwetter,Niesel,Schnee,Schnee,Schnee
3,MAE-HAM,BOX,20G,0,0,0,1,NaN,15492.0,BRV13837Z,...,January,HAS,U,Regen,Niesel,Unwetter,Niesel,Schnee,Schnee,Schnee
4,MAE-HAM,BOX,40GH,0,0,0,1,NaN,5778.0,BRV13837Z,...,January,MRK,U,Regen,Niesel,Unwetter,Niesel,Schnee,Schnee,Schnee
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276146,TFG,NaN,40GH,0,0,0,1,NaN,5510.0,NaN,...,December,ONE,U,Regen,Regen,Unwetter,Regen,Regen,Regen,Regen
276147,TFG,NaN,40GH,0,0,0,1,NaN,8286.0,NaN,...,December,SEL,U,Regen,Regen,Unwetter,Regen,Regen,Regen,Regen
276148,TFG,NaN,40GH,0,0,0,1,NaN,3802.0,NaN,...,December,FFA,U,Regen,Regen,Unwetter,Regen,Regen,Regen,Regen
276149,TFG,NaN,40GH,0,0,0,1,NaN,5662.0,NaN,...,December,HAM,U,Regen,Regen,Unwetter,Regen,Regen,Regen,Regen


# Container Parameter Distributions

In [4]:
unittypes = pd.read_csv(f'{os.getcwd()}/unittype.csv')

# Create a new column 'unit_type' in df and do a join of container type onto unittype and return category
df['unit_type'] = df['container_type'].map(unittypes.set_index('Unittype')['Category'])
df['unit_type'] = df['unit_type'].fillna('Unknown')
df['unit_type'] = df['unit_type'].replace('', 'Unknown')

df['unit_type'].value_counts()
print(P(df, 'unit_type'))

{"40' Container": 0.516474682329595, 'Wechselbrücke': 0.2485524224065819, "20' Container": 0.17511071841130396, 'Trailer': 0.031486396935010194, "30' Container": 0.017827203233013823, "45' Container": 0.010544955477257008, 'Unknown': 3.6212072380690276e-06}


In [ ]:
# First, let's calculate P(unit_type | dangerous_goods)
# This shows the distribution of container types given dangerous goods status
p_unit_type_given_dg = {}
for dg in [0, 1]:  # 0 = non-dangerous, 1 = dangerous
    # Filter dataframe by dangerous_goods value
    filtered_df = df[df['dangerous_goods'] == dg]
    # Calculate probability distribution within this filtered dataset
    p_unit_type_given_dg[dg] = P(filtered_df, 'unit_type')
    
print(f"P(unit_type | dangerous_goods=0):")
print(p_unit_type_given_dg[0])
print(f"\nP(unit_type | dangerous_goods=1):")
print(p_unit_type_given_dg[1])

# Now calculate P(dangerous_goods | unit_type)
# This shows the probability of dangerous goods for each container type
p_dg_given_unit_type = {}
for unit_type in df['unit_type'].unique():
    # Filter dataframe by unit_type
    filtered_df = df[df['unit_type'] == unit_type]
    # Calculate probability distribution within this filtered dataset
    p_dg_given_unit_type[unit_type] = P(filtered_df, 'dangerous_goods')

print("\nP(dangerous_goods | unit_type):")
for unit_type, probs in p_dg_given_unit_type.items():
    print(f"{unit_type}: {probs}")
# remove all entries where the container type does not start with '20' or '40'
df = df[df['unit_type'].str.startswith(('20', '40', 'Wechselbrücke', 'Trailer')).copy()]
print('Container type distribution')
print(P(df, 'unit_type'))

# return probs for dangerous goods
print('Dangerous goods distribution')
p_dg = P(df, 'dangerous_goods')
print(p_dg)

# dangerous goods have 15 places in the yard, reefer containers have 10 places in the yard
# reefer containers have 2/3 of the places in the yard
print('Reefer container distribution')
p_ref = {1-p_dg[1]*2/3, p_dg[1]*2/3}
print(p_ref)

#TODO: create distribution of container heights
#TODO: create distribution of container weights

P(unit_type | dangerous_goods=0):
{"40' Container": 0.5223409956903831, 'Wechselbrücke': 0.2481370267313721, "20' Container": 0.17513784166746202, 'Trailer': 0.0296020028339219, "30' Container": 0.01408885005910122, "45' Container": 0.010689612134471796, 'Unknown': 3.6708832879367434e-06}

P(unit_type | dangerous_goods=1):
{"30' Container": 0.29033984479529035, 'Wechselbrücke': 0.2788332887342788, "20' Container": 0.17313352956917313, 'Trailer': 0.16885202033716884, "40' Container": 0.08884131656408883}

P(dangerous_goods | unit_type):
20' Container: {0: 0.9866203445209587, 1: 0.013379655479041297}
40' Container: {0: 0.997672217353199, 1: 0.0023277826468010515}
Wechselbrücke: {0: 0.9848189049797488, 1: 0.015181095020251173}
45' Container: {0: 1.0}
30' Container: {0: 0.7796059313426772, 1: 0.22039406865732278}
Trailer: {0: 0.9274295572167912, 1: 0.07257044278320875}
Unknown: {0: 1.0}
Container type distribution
{"40' Container": 0.5315580567616421, 'Wechselbrücke': 0.2558112666082776, "

In [ ]:
#TODO: Distribution of truck arrival times
#TODO: Distribution of train arrival times
#TODO: Distribution of delay times
#TODO: Duration of stay for trains in yard
#TODO: Average duration of stay for trucks in yard